In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.isna().sum()

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the train_dataset and 20% of the data as the test_dataset.

pop off the "expenses" column from these datasets to create new datasets called train_labels and test_labels. Use these labels when training your model.

Create a model and train it with the train_dataset. Run the final cell in this notebook to check your model. The final cell will use the unseen test_dataset to check how well the model generalizes.

To pass the challenge, model.evaluate must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the test_dataset and graph the results.

In [ ]:
# Data Preparing

CATEGORICAL_COLUMNS = ['sex','smoker','region']

# converting string values to numerical data
dataset = pd.get_dummies(dataset, columns = CATEGORICAL_COLUMNS)

dataset


In [ ]:

# Dataset preparation before training and testing the model

train_size = 0.8
test_size = 0.2
# train 80% test 20%
raw_train_data = dataset.iloc[: int(len(dataset)*train_size)]
raw_test_data = dataset.iloc[int(len(dataset)*train_size) : int(len(dataset)* (test_size + train_size))]

# exclusion of 'expenses' column
train_labels = raw_train_data['expenses']
train_dataset = raw_train_data.drop(columns = 'expenses')

test_labels = raw_test_data['expenses']
test_dataset = raw_test_data.drop(columns = 'expenses')


The number of epochs is a hyperparameter that determines how many times the model will go through the entire dataset. It's important to find an appropriate number of epochs to prevent underfitting (too few epochs) or overfitting (too many epochs).


he batch size is a hyperparameter that determines the number of samples in each batch. Common batch sizes are powers of 2, such as 32, 64, or 128, but it can be adjusted based on the available memory and computational resources.

If you have 1000 training samples and a batch size of 64, it will take 16 batches to complete one epoch.

In [ ]:
# Create the model

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='relu')
  ])

# Compile the model
optimizer="Adam"
model.compile(loss='mae',
              optimizer=optimizer,
              metrics=['mae','mse'])

# Train the model
model.fit(train_dataset, train_labels, epochs=100, batch_size=32,verbose='auto',
    callbacks=None,)


In [ ]:
model.summary()

In [ ]:
## RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
